In [1]:
# from lib.alpwrapper import Database
from lib.database import Database
from lib.schema import Schema
from lib.display import Display

In [2]:
Display.header('FeMFit', 'h1')
db = Database(Schema.FEMFIT)
Display.header('Participants', 'h2')
femfit_participants_df = db.get_participants()
Display.frame(femfit_participants_df, 5)
Display.header('Responses', 'h2')
femfit_responses_df = db.get_responses()
Display.frame(femfit_responses_df, 5)

Display.header('Broad consent', 'h1')
db = Database(Schema.BROAD_CONSENT)
Display.header('Participants', 'h2')
bc_participants_df = db.get_participants()
Display.frame(bc_participants_df, 5)
Display.header('Responses', 'h2')
bc_responses_df = db.get_responses()
Display.frame(bc_responses_df, 5)

,ALP_ID,EXTERNAL_ID,STATUS,START_DATE,END_DATE
0,12255,Participant_26,enrolled,2022-03-16,NaT
1,12373,Participant_13,enrolled,2022-03-15,NaT
2,10116,Participant_04,enrolled,2022-01-14,NaT
3,10158,Participant_07,enrolled,2022-01-27,NaT
4,12558,Participant_33,enrolled,2022-03-22,NaT


,ALP_ID,VERSION,AUTHORED,QUESTIONNAIRE,LINK_ID,VALUE,VALUECODING_CODE,LANGUAGE,TEXT,QUESTIONNAIRE_ID
0,8557,1.0.0,2022-01-07,Q2_physical_wellbeing_and_activity,satisfaction_ physical_activity,NaN,moving_4_good,de,"Wie gut kÃ¶nnen Sie sich bewegen (z.B. gehen, Treppen steigen)?",aNo7a6cEAVLIctP7GSGhPaUieIhAhor+K7Dtvzu0rxNW6y8+O4n/SRVF+StNOYE0dKnlEfj8C5xlR7pxL6CWJCmj2lmgoVcuty33F8uZ4hXrv8rduc3etNnSMBxeaZgQpWAKJ7f09gMZUb0C7zpIJi40P39fafyTmOOV54U7XMqNSh9FiBy+WLBc5ex6+6N1ULktuDutQAcXcR2m2Hw8koBt26Yd+Uo81SkwpHxYjOUA1ZANTkSnjTZqXJma/LT1+7klJjUa6enH5A7tTp93NOvjJ335cY6pXIlqOmWwXKoOTWZ4YVlVnTr51ZAJauvJyr6cee7yIgKNeItCwqRPuA==
1,8557,1.0.0,2022-01-07,Q2_physical_wellbeing_and_activity,wearing_fitbit_yes_no,NaN,yes,de,"Haben Sie das Fitbit-Armband kontinuierlich getragen (besonders auch nachts, wÃ¤hrend Bewegung, etc)?",aNo7a6cEAVLIctP7GSGhPaUieIhAhor+K7Dtvzu0rxNW6y8+O4n/SRVF+StNOYE0dKnlEfj8C5xlR7pxL6CWJCmj2lmgoVcuty33F8uZ4hXrv8rduc3etNnSMBxeaZgQpWAKJ7f09gMZUb0C7zpIJi40P39fafyTmOOV54U7XMqNSh9FiBy+WLBc5ex6+6N1ULktuDutQAcXcR2m2Hw8koBt26Yd+Uo81SkwpHxYjOUA1ZANTkSnjTZqXJma/LT1+7klJjUa6enH5A7tTp93NOvjJ335cY6pXIlqOmWwXKoOTWZ4YVlVnTr51ZAJauvJyr6cee7yIgKNeItCwqRPuA==
2,8557,1.0.0,2022-01-07,Q2_physical_wellbeing_and_activity,satisfaction_sleep,NaN,satisfaction_2_unhappy,de,Wie zufrieden sind Sie mit Ihrem Schlaf?,aNo7a6cEAVLIctP7GSGhPaUieIhAhor+K7Dtvzu0rxNW6y8+O4n/SRVF+StNOYE0dKnlEfj8C5xlR7pxL6CWJCmj2lmgoVcuty33F8uZ4hXrv8rduc3etNnSMBxeaZgQpWAKJ7f09gMZUb0C7zpIJi40P39fafyTmOOV54U7XMqNSh9FiBy+WLBc5ex6+6N1ULktuDutQAcXcR2m2Hw8koBt26Yd+Uo81SkwpHxYjOUA1ZANTkSnjTZqXJma/LT1+7klJjUa6enH5A7tTp93NOvjJ335cY6pXIlqOmWwXKoOTWZ4YVlVnTr51ZAJauvJyr6cee7yIgKNeItCwqRPuA==
3,8557,1.0.0,2022-01-07,Q2_physical_wellbeing_and_activity,satisfaction_health,NaN,satisfaction_2_unhappy,de,Wie zufrieden sind Sie mit Ihrer Gesundheit?,aNo7a6cEAVLIctP7GSGhPaUieIhAhor+K7Dtvzu0rxNW6y8+O4n/SRVF+StNOYE0dKnlEfj8C5xlR7pxL6CWJCmj2lmgoVcuty33F8uZ4hXrv8rduc3etNnSMBxeaZgQpWAKJ7f09gMZUb0C7zpIJi40P39fafyTmOOV54U7XMqNSh9FiBy+WLBc5ex6+6N1ULktuDutQAcXcR2m2Hw8koBt26Yd+Uo81SkwpHxYjOUA1ZANTkSnjTZqXJma/LT1+7klJjUa6enH5A7tTp93NOvjJ335cY6pXIlqOmWwXKoOTWZ4YVlVnTr51ZAJauvJyr6cee7yIgKNeItCwqRPuA==
4,8557,1.0.0,2022-01-07,Q2_physical_wellbeing_and_activity,satisfaction_feelings,NaN,frequency_3_occasionally,de,"Wie hÃ¤ufig haben Sie negative GefÃ¼hle wie Traurigkeit, Verzweiflung oder Angst?",aNo7a6cEAVLIctP7GSGhPaUieIhAhor+K7Dtvzu0rxNW6y8+O4n/SRVF+StNOYE0dKnlEfj8C5xlR7pxL6CWJCmj2lmgoVcuty33F8uZ4hXrv8rduc3etNnSMBxeaZgQpWAKJ7f09gMZUb0C7zpIJi40P39fafyTmOOV54U7XMqNSh9FiBy+WLBc5ex6+6N1ULktuDutQAcXcR2m2Hw8koBt26Yd+Uo81SkwpHxYjOUA1ZANTkSnjTZqXJma/LT1+7klJjUa6enH5A7tTp93NOvjJ335cY6pXIlqOmWwXKoOTWZ4YVlVnTr51ZAJauvJyr6cee7yIgKNeItCwqRPuA==


,ALP_ID,EXTERNAL_ID,STATUS,START_DATE,END_DATE
0,699,anonymous,enrolled,2022-03-25,NaT
1,731,anonymous,enrolled,2022-03-29,NaT
2,729,anonymous,enrolled,2022-04-04,NaT
3,715,anonymous,enrolled,2022-03-28,NaT
4,723,anonymous,enrolled,2022-03-28,NaT


,ALP_ID,VERSION,AUTHORED,QUESTIONNAIRE,LINK_ID,VALUE,VALUECODING_CODE,LANGUAGE,TEXT,QUESTIONNAIRE_ID
0,697,1.0.0,2022-03-25,Q1_personal_info,when_participant_born,2000,NaN,de,In welchem Jahr sind Sie geboren?,ofracPQyDuZZ_bXsGq34mFwowQxyGRyaRp4FIX4ETgp-rKV8W1FkmvawW6EPp6RqgOClBN8Chd25Ceq0L82Xqwnqxhu_vgk9zwPIoV8eWqFe9F7xXvHM0St405QLExLrPBXTEkRYgmXCA0aAS_wjVMu3lWArgkribFTQotyIjVMtESJrLANDZhyjz94NWjLSAz8q5Sl2GuuxsAhHTNYscwZxBKNzwKny3sUdVUv2u73_d_8epD8zT-pxZ4tyEFoZUHKvmZMD2wlXPoSUfCvFxu6HlOz_qSyi_boDceM9Zu3MuA3F_13F9qhU2ICZOPXNiK9OYBpMAQEuS8i9G5HusA==
1,697,1.0.0,2022-03-25,Q1_personal_info,participant_preexisting_conditions,NaN,preexisting_2_pcos,de,Haben Sie eine oder mehrere dieser Vorerkrankungen?,ofracPQyDuZZ_bXsGq34mFwowQxyGRyaRp4FIX4ETgp-rKV8W1FkmvawW6EPp6RqgOClBN8Chd25Ceq0L82Xqwnqxhu_vgk9zwPIoV8eWqFe9F7xXvHM0St405QLExLrPBXTEkRYgmXCA0aAS_wjVMu3lWArgkribFTQotyIjVMtESJrLANDZhyjz94NWjLSAz8q5Sl2GuuxsAhHTNYscwZxBKNzwKny3sUdVUv2u73_d_8epD8zT-pxZ4tyEFoZUHKvmZMD2wlXPoSUfCvFxu6HlOz_qSyi_boDceM9Zu3MuA3F_13F9qhU2ICZOPXNiK9OYBpMAQEuS8i9G5HusA==
2,697,1.0.0,2022-03-25,Q1_personal_info,participant_cycle_regularity,NaN,yes,de,Ist ihr Zyklus regelmÃ¤Ãig? (alle 28 +/- 5 Tage),ofracPQyDuZZ_bXsGq34mFwowQxyGRyaRp4FIX4ETgp-rKV8W1FkmvawW6EPp6RqgOClBN8Chd25Ceq0L82Xqwnqxhu_vgk9zwPIoV8eWqFe9F7xXvHM0St405QLExLrPBXTEkRYgmXCA0aAS_wjVMu3lWArgkribFTQotyIjVMtESJrLANDZhyjz94NWjLSAz8q5Sl2GuuxsAhHTNYscwZxBKNzwKny3sUdVUv2u73_d_8epD8zT-pxZ4tyEFoZUHKvmZMD2wlXPoSUfCvFxu6HlOz_qSyi_boDceM9Zu3MuA3F_13F9qhU2ICZOPXNiK9OYBpMAQEuS8i9G5HusA==
3,697,1.0.0,2022-03-25,Q1_personal_info,participant_gender,NaN,gender_w,de,Welchem Geschlecht ordnen Sie sich zu?,ofracPQyDuZZ_bXsGq34mFwowQxyGRyaRp4FIX4ETgp-rKV8W1FkmvawW6EPp6RqgOClBN8Chd25Ceq0L82Xqwnqxhu_vgk9zwPIoV8eWqFe9F7xXvHM0St405QLExLrPBXTEkRYgmXCA0aAS_wjVMu3lWArgkribFTQotyIjVMtESJrLANDZhyjz94NWjLSAz8q5Sl2GuuxsAhHTNYscwZxBKNzwKny3sUdVUv2u73_d_8epD8zT-pxZ4tyEFoZUHKvmZMD2wlXPoSUfCvFxu6HlOz_qSyi_boDceM9Zu3MuA3F_13F9qhU2ICZOPXNiK9OYBpMAQEuS8i9G5HusA==
4,697,1.0.0,2022-03-25,Q1_personal_info,participant_hormonal_contraception,NaN,no,de,"Nutzen Sie derzeit hormonelle VerhÃ¼tung? (z.B. Pille, Hormonspirale, Ring, Implantat)",ofracPQyDuZZ_bXsGq34mFwowQxyGRyaRp4FIX4ETgp-rKV8W1FkmvawW6EPp6RqgOClBN8Chd25Ceq0L82Xqwnqxhu_vgk9zwPIoV8eWqFe9F7xXvHM0St405QLExLrPBXTEkRYgmXCA0aAS_wjVMu3lWArgkribFTQotyIjVMtESJrLANDZhyjz94NWjLSAz8q5Sl2GuuxsAhHTNYscwZxBKNzwKny3sUdVUv2u73_d_8epD8zT-pxZ4tyEFoZUHKvmZMD2wlXPoSUfCvFxu6HlOz_qSyi_boDceM9Zu3MuA3F_13F9qhU2ICZOPXNiK9OYBpMAQEuS8i9G5HusA==


In [3]:
# from lib.disk import Disk

# disk = Disk('FEMFIT')
# disk.save_participants(femfit_participants_df)
# disk.save_responses(femfit_responses_df)
# disk.save_wearbles(wearables_df)

In [5]:
import unittest
from pandas.testing import assert_frame_equal

class BroadConsentTestCase(unittest.TestCase):
    def setUp(self):
        db = Database(Schema.FEMFIT)
        self.femfit_participants_df = db.get_participants()
        self.femfit_responses_df = db.get_responses()
        db = Database(Schema.BROAD_CONSENT)
        self.bc_participants_df = db.get_participants()
        self.bc_responses_df = db.get_responses()

    def test_data_of_femfit_user(self):
        external_id = 'ohad_test_05'
        femfit_alp_id = 732

        participant_row = self.femfit_participants_df[femfit_participants_df['EXTERNAL_ID'] == external_id].iloc[0]
        self.assertEqual(participant_row["ALP_ID"], femfit_alp_id, f'external id of {external_id} is {femfit_alp_id}')
            
        response_row = self.femfit_responses_df[
            (self.femfit_responses_df['LINK_ID'] == 'participant_last_bleeding') & 
            (self.femfit_responses_df['ALP_ID'] == femfit_alp_id)].iloc[0]
        self.assertEqual(response_row["VALUE"], '2022-03-18', 'Last bleeding date')
        self.assertEqual(response_row["AUTHORED"].strftime("%d.%m.%Y"), '29.03.2022', 'Authored date')


    def test_same_user_in_both_silos(self):
        femfit_alp_id = 732
        bc_alp_id = 731
        df1 = self.femfit_responses_df[femfit_responses_df['ALP_ID'] == femfit_alp_id] \
            .sort_values(by=['AUTHORED', 'LINK_ID']) \
            .drop(columns=['ALP_ID', 'QUESTIONNAIRE_ID']).reset_index(drop=True)
        df2 = self.bc_responses_df[bc_responses_df['ALP_ID'] == bc_alp_id] \
            .sort_values(by=['AUTHORED', 'LINK_ID']) \
            .drop(columns=['ALP_ID', 'QUESTIONNAIRE_ID']).reset_index(drop=True)
        assert_frame_equal(df1, df2, 'Not the same user')

        
def suite():
    suite = unittest.TestSuite()
    suite.addTest(BroadConsentTestCase('test_data_of_femfit_user'))
    suite.addTest(BroadConsentTestCase('test_same_user_in_both_silos'))
    return suite

if __name__ == '__main__':
    runner = unittest.TextTestRunner()
    runner.run(suite())

..
----------------------------------------------------------------------
Ran 2 tests in 0.086s

OK


In [6]:
import pandas as pd

FEMFIT_ALP_ID = 725
external_id = femfit_participants_df[femfit_participants_df['ALP_ID'] == FEMFIT_ALP_ID]['EXTERNAL_ID'].iloc[0]
print(f'FeMFit ALP ID\t: {FEMFIT_ALP_ID}')
print(f'External ID\t: {external_id}')

df1 = femfit_responses_df[femfit_responses_df['ALP_ID'] == FEMFIT_ALP_ID][['ALP_ID', 'QUESTIONNAIRE', 'LINK_ID', 'VALUECODING_CODE', 'VALUE']] \
    .rename(columns={'VALUECODING_CODE' : f'VALUECODING_CODE ({FEMFIT_ALP_ID})', 'VALUE' : f'VALUE ({FEMFIT_ALP_ID})'}) \
    .fillna('').reset_index(drop=True)

BC_ALP_ID = 729
df2 = bc_responses_df[bc_responses_df['ALP_ID'] == BC_ALP_ID][['ALP_ID', 'QUESTIONNAIRE', 'LINK_ID', 'VALUECODING_CODE', 'VALUE']] \
    .rename(columns={'VALUECODING_CODE' : f'VALUECODING_CODE ({BC_ALP_ID})', 'VALUE' : f'VALUE ({BC_ALP_ID})'}) \
    .fillna('').reset_index(drop=True)

columns = ['QUESTIONNAIRE', 'LINK_ID', f'VALUECODING_CODE ({FEMFIT_ALP_ID})', f'VALUECODING_CODE ({BC_ALP_ID})', f'VALUE ({FEMFIT_ALP_ID})', f'VALUE ({BC_ALP_ID})']
df = pd.merge(df1, df2, how='left', on=['QUESTIONNAIRE', 'LINK_ID']) \
    .sort_values(by=['QUESTIONNAIRE', 'LINK_ID']) \
    .drop(columns=['ALP_ID_x', 'ALP_ID_y'])[columns]

Display.header('Comparison between FeMFit and broad consent', 'h3')
Display.frame(df)

FeMFit ALP ID	: 725
External ID	: ohad_test_13a


,QUESTIONNAIRE,LINK_ID,VALUECODING_CODE (725),VALUECODING_CODE (729),VALUE (725),VALUE (729)
18,Q1_personal_info,participant_cycle_regularity,yes,NaN,,NaN
20,Q1_personal_info,participant_gender,gender_w,NaN,,NaN
19,Q1_personal_info,participant_hormonal_contraception,no,NaN,,NaN
17,Q1_personal_info,participant_last_bleeding,,NaN,2022-03-27,NaN
21,Q1_personal_info,when_participant_born,,NaN,2000,NaN
16,Q2_physical_wellbeing_and_activity,how_long_fitbit_off,duration_3_3h,NaN,,NaN
15,Q2_physical_wellbeing_and_activity,wearing_fitbit_yes_no,no,NaN,,NaN
10,Q3_cycle_dependent,anger,yes,yes,,
9,Q3_cycle_dependent,anger_severity,,,2,2
11,Q3_cycle_dependent,anxiety,no,no,,


In [7]:
from lib.studies.broad_consent import BroadConsent 
from lib.participant_builder import ParticipantsBuilder

program = BroadConsent()
print('Study:', program.name())
print('Last donation:', bc_responses_df['AUTHORED'].max().strftime("%Y-%m-%d"))

builder = ParticipantsBuilder(program, bc_participants_df, bc_responses_df)
df = builder \
    .add_last_donation() \
    .add_all_submissions() \
    .get() \
    .sort_values(by=['LAST_DONATION'], ascending=False)

Display.frame(df)

Study: BroadConsent
Last donation: 2022-06-20


,ALP_ID,EXTERNAL_ID,STATUS,START_DATE,END_DATE,ENROLED_ON,LAST_DONATION,all,FeMFit_Q1,FeMFit_Q2,FeMFit_Q3,FeMFit_Q4,FeMFit_Q5
9,713,anonymous,enrolled,2022-03-28 00:00:00,NaN,2022-03-28,2022-06-20,20,1,4,15,0,0
1,731,anonymous,enrolled,2022-03-29 00:00:00,NaN,2022-03-29,2022-03-29,1,1,0,0,0,0
2,729,anonymous,enrolled,2022-04-04 00:00:00,NaN,NaT,2022-03-29,1,0,0,1,0,0
0,699,anonymous,enrolled,2022-03-25 00:00:00,NaN,2022-03-25,2022-03-25,3,1,0,1,0,1
8,697,anonymous,enrolled,2022-03-25 00:00:00,NaN,2022-03-25,2022-03-25,2,1,0,1,0,0
12,701,anonymous,enrolled,2022-03-25 00:00:00,NaN,2022-03-25,2022-03-25,2,1,0,1,0,0
3,715,anonymous,enrolled,2022-03-28 00:00:00,NaN,NaT,NaT,0,0,0,0,0,0
4,723,anonymous,enrolled,2022-03-28 00:00:00,NaN,NaT,NaT,0,0,0,0,0,0
5,724,anonymous,withdrawn,2022-03-28 15:57:19,2022-04-04 15:59:29,NaT,NaT,0,0,0,0,0,0
6,720,anonymous,withdrawn,2022-03-28 14:50:29,2022-03-28 15:30:22,NaT,NaT,0,0,0,0,0,0
